# Data loading

In [ ]:
from zipfile import ZipFile
train = './train.zip'

with ZipFile(train, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [ ]:
from zipfile import ZipFile
test = './test.zip'

with ZipFile(test, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from torchvision.io import read_image
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import os
import torch.nn.functional as F
import torchvision

# Data preprossing

In [12]:
# Data preprocessing
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [13]:
train_dir = './train'
test_dir = './test'

In [14]:
# Create datasets
train_dataset = ImageFolder(train_dir, transform=transform)
test_dataset = ImageFolder(test_dir, transform=transform)

# Training

In [25]:
# Hyperparameters
batch_size = 64
num_epochs = 10
learning_rate = 0.001

In [26]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(32 * 14 * 14, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, 32 * 14 * 14)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
model = CNN()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [50]:
def train(model, train_loader, optimizer, criterion):
    train_loss = 0

    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        #print(outputs.shape)
        outputs = torch.squeeze(outputs)
        #print(outputs.shape)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()

        train_loss += loss.item()*inputs.size(0)

    return train_loss

def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            #print(outputs.shape)
            outputs = torch.squeeze(outputs)
            #print(outputs.shape)
            predicted = (outputs > 0).int()  # Convert logits to binary predictions
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    print(f'total: {total}, correct: {correct}')
    return accuracy


In [52]:
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, optimizer, criterion)
    accuracy = evaluate(model, test_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Test Accuracy: {accuracy:.4f}")

# Save the trained model
torch.save(model.state_dict(), 'binary_classifier_cnn.pth')

total: 2080, correct: 2078
Epoch 1/10, Test Accuracy: 0.9990
total: 2080, correct: 2079
Epoch 2/10, Test Accuracy: 0.9995
total: 2080, correct: 2079
Epoch 3/10, Test Accuracy: 0.9995
total: 2080, correct: 2079
Epoch 4/10, Test Accuracy: 0.9995
total: 2080, correct: 2079
Epoch 5/10, Test Accuracy: 0.9995
total: 2080, correct: 2079
Epoch 6/10, Test Accuracy: 0.9995
total: 2080, correct: 2079
Epoch 7/10, Test Accuracy: 0.9995
total: 2080, correct: 2079
Epoch 8/10, Test Accuracy: 0.9995
total: 2080, correct: 2078
Epoch 9/10, Test Accuracy: 0.9990
total: 2080, correct: 2079
Epoch 10/10, Test Accuracy: 0.9995
